In [1]:
import warnings

from warnings import simplefilter

from sklearn.exceptions import DataConversionWarning

simplefilter(action='ignore', category=FutureWarning)

warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle as pkl
import math
import csv
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import NearMiss
from collections import Counter

In [3]:
#flight_data = pd.read_csv(r'C:\Users\badri\Desktop\ML-Project\Flight Data\Flight_Data.csv')
from pathlib import Path

# 1) Define your project root (where you launched the script / notebook)
BASE_DIR = Path().resolve()      # or: Path(__file__).parent if in a .py module

# 2) Point at your data folder
DATA_DIR = BASE_DIR / 'data'

# 3) Build each file’s path dynamically
flight_csv   = DATA_DIR / 'filtered_flights.csv'
weather_csv  = DATA_DIR / 'Weather' / 'Weather_Data.csv'
final_csv   = DATA_DIR / 'Flight_Weather.csv'

# 4) Read them with pandas
#flight_data  = pd.read_csv(flight_csv)
#weather_data = pd.read_csv(weather_csv)
dataframe = pd.read_csv(final_csv)

In [4]:
#!pip install -U -q PyDrive
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials

In [5]:
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
##drive = GoogleDrive(gauth)

In [6]:
#def get_dataframe(link,filename) :

  #fluff, id = link.split('=')

  #downloaded = drive.CreateFile({'id':id}) 

  #downloaded.GetContentFile(filename)

  #dataframe = pd.read_csv(filename)

  #data_label_encoder = preprocessing.LabelEncoder()

  #encoded_list = data_label_encoder.fit_transform(dataframe['Airport'])

  #dataframe['EncodedAirport'] = encoded_list

  #return dataframe

In [7]:
#dataframe = get_dataframe('https://drive.google.com/open?id=17DbBIxtBcBM9Q4MWbeF46ml8WNv_ATKl','Flight_Weather.csv')

In [8]:
y = dataframe[['ArrDel15','ArrDelayMinutes']]

dataframe = dataframe.drop(['FlightDate','Date','Airport','ArrDelayMinutes'],axis = 1)

X = dataframe

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [10]:
X_train_regressor = X_train[X_train['ArrDel15'] == 1].drop(['ArrDel15'] , axis = 1)

X_test_regressor = X_test[X_test['ArrDel15'] == 1].drop(['ArrDel15'] , axis = 1)

X_train_classifier = X_train.drop(['ArrDel15'] , axis = 1)

X_test_classifier = X_test.drop(['ArrDel15'] , axis = 1)

y_train_classifier = y_train.drop(['ArrDelayMinutes'] , axis = 1)

y_test_classifier = y_test.drop(['ArrDelayMinutes'] , axis = 1)

y_train_regressor = y_train[y_train['ArrDel15'] == 1].drop(['ArrDel15'] , axis = 1)

y_test_regressor = y_test[y_test['ArrDel15'] == 1].drop(['ArrDel15'] , axis = 1)

In [11]:
print(" Original DataSet Shape : ",Counter(y_train_classifier.T.squeeze()))

ros = RandomOverSampler(random_state = 42)

X_ros, y_ros = ros.fit_resample(X_train_classifier, y_train_classifier.T.squeeze())

print(" RandomOverSampled DataSet Shape : ",Counter(y_ros))

smote = SMOTE(random_state = 42)

X_smote, y_smote = smote.fit_resample(X_train_classifier, y_train_classifier.T.squeeze())

print(" SMOTE DataSet Shape : ",Counter(y_smote))

 Original DataSet Shape :  Counter({0.0: 3007264, 1.0: 728299})
 RandomOverSampled DataSet Shape :  Counter({0.0: 3007264, 1.0: 3007264})
 SMOTE DataSet Shape :  Counter({0.0: 3007264, 1.0: 3007264})


In [12]:
print(" Original DataSet Shape : ",Counter(y_train_classifier.T.squeeze()))

rus = RandomUnderSampler(random_state = 42)

X_rus, y_rus = rus.fit_resample(X_train_classifier, y_train_classifier.T.squeeze())

print(" RandomUnderampled DataSet Shape : ",Counter(y_rus))

nm = NearMiss()

X_nm, y_nm = nm.fit_resample(X_train_classifier, y_train_classifier.T.squeeze())

print(" NearMiss DataSet Shape : ",Counter(y_nm))

 Original DataSet Shape :  Counter({0.0: 3007264, 1.0: 728299})
 RandomUnderampled DataSet Shape :  Counter({0.0: 728299, 1.0: 728299})
 NearMiss DataSet Shape :  Counter({0.0: 728299, 1.0: 728299})


In [13]:
clf = RandomForestClassifier(n_estimators = 100)

In [14]:
clf.fit(X_train_classifier,y_train_classifier)

y_pred = clf.predict(X_test_classifier)

print(" Classification Report ")

print("\n\n")

print(metrics.classification_report(y_test_classifier,y_pred))

 Classification Report 



              precision    recall  f1-score   support

         0.0       0.95      0.98      0.97   1002560
         1.0       0.92      0.77      0.84    242628

    accuracy                           0.94   1245188
   macro avg       0.93      0.88      0.90   1245188
weighted avg       0.94      0.94      0.94   1245188



In [15]:
clf.fit(X_smote,y_smote)

y_pred_smote = clf.predict(X_test_classifier)

print(" Classification Report")

print("\n\n")

print(metrics.classification_report(y_test_classifier,y_pred_smote))

 Classification Report



              precision    recall  f1-score   support

         0.0       0.95      0.97      0.96   1002560
         1.0       0.88      0.78      0.83    242628

    accuracy                           0.94   1245188
   macro avg       0.91      0.88      0.89   1245188
weighted avg       0.93      0.94      0.93   1245188



In [16]:
rfr = RandomForestRegressor()

In [17]:
rfr.fit(X_train_regressor,y_train_regressor)

y_pred = rfr.predict(X_test_regressor)

In [18]:
print(" Mean Absolute Error : ", metrics.mean_absolute_error(y_test_regressor, y_pred))  

print(" Mean Squared Error : ", metrics.mean_squared_error(y_test_regressor, y_pred))  

print(" Root Mean Squared Error : ", np.sqrt(metrics.mean_squared_error(y_test_regressor, y_pred)))

print(" R2 Score : ",metrics.r2_score(y_test_regressor,y_pred))

 Mean Absolute Error :  5.597908773925515
 Mean Squared Error :  94.55720554305357
 Root Mean Squared Error :  9.724052938104233
 R2 Score :  0.9800383608927881


In [19]:
Y = y_test_regressor

In [20]:
X_test_regression_analysis =  X_test_regressor

In [21]:
X_test_regression_analysis['ArrDelayMinutes'] = y_test_regressor

In [22]:
Y_15_100= Y.loc[Y['ArrDelayMinutes'].isin(list(range(15,101)))]

Y_100_200 = Y.loc[Y['ArrDelayMinutes'].isin(list(range(101,201)))]

Y_200_500 = Y.loc[Y['ArrDelayMinutes'].isin(list(range(201,501)))]

Y_500_1000 = Y.loc[Y['ArrDelayMinutes'].isin(list(range(501,1001)))]

Y_1000_2000 = Y.loc[Y['ArrDelayMinutes'].isin(list(range(1001,2001)))]

Y_15_100_1 = Y_15_100['ArrDelayMinutes']

Y_100_200_1 = Y_100_200['ArrDelayMinutes']

Y_200_500_1 = Y_200_500['ArrDelayMinutes']

Y_500_1000_1 = Y_500_1000['ArrDelayMinutes']

Y_1000_2000_1 = Y_1000_2000['ArrDelayMinutes']


X = X_test_regression_analysis

X_15_100 = X.loc[X['ArrDelayMinutes'].isin(list(range(15,101)))]

X_100_200 = X.loc[X['ArrDelayMinutes'].isin(list(range(101,201)))]

X_200_500 = X.loc[X['ArrDelayMinutes'].isin(list(range(201,501)))]

X_500_1000 = X.loc[X['ArrDelayMinutes'].isin(list(range(501,1001)))]

X_1000_2000 = X.loc[X['ArrDelayMinutes'].isin(list(range(1001,2001)))]

X_15_100_1 = X_15_100.drop(['ArrDelayMinutes'],axis = 1)

X_100_200_1 = X_100_200.drop(['ArrDelayMinutes'],axis = 1)

X_200_500_1 = X_200_500.drop(['ArrDelayMinutes'],axis = 1)

X_500_1000_1 = X_500_1000.drop(['ArrDelayMinutes'],axis = 1)

X_1000_2000_1 = X_1000_2000.drop(['ArrDelayMinutes'],axis = 1)

In [23]:
def scores_rfr(X,Y,Z):

  for i,j,k in zip(X,Y,Z):
    
    print(k)

    y_pred = rfr.predict(i)

    print(" Mean Absolute Error : ", metrics.mean_absolute_error(j, y_pred))  

    print(" Mean Squared Error : ", metrics.mean_squared_error(j, y_pred))  

    print(" Root Mean Squared Error : ", np.sqrt(metrics.mean_squared_error(j, y_pred)))

    print(" R2 Score : ",metrics.r2_score(j,y_pred))

    print("\n\n")

In [24]:
X_list = [X_15_100_1,X_100_200_1,X_200_500_1 ,X_500_1000_1 ,X_1000_2000_1]

Y_list = [Y_15_100_1,Y_100_200_1,Y_200_500_1,Y_500_1000_1 ,Y_1000_2000_1]

Z_list = ['15_100','100_200','200_500','500_1000','1000_2000']

In [25]:
scores_rfr(X_list,Y_list,Z_list)

15_100
 Mean Absolute Error :  4.6775862085784246
 Mean Squared Error :  56.90359515134665
 Root Mean Squared Error :  7.543447166338918
 R2 Score :  0.8834964207187026



100_200
 Mean Absolute Error :  9.550235968949647
 Mean Squared Error :  237.08268557738546
 Root Mean Squared Error :  15.39748958685751
 R2 Score :  0.6755454219786254



200_500
 Mean Absolute Error :  14.207391247889884
 Mean Squared Error :  511.3815061160888
 Root Mean Squared Error :  22.613745954973687
 R2 Score :  0.8791914791865294



500_1000
 Mean Absolute Error :  14.554718309859155
 Mean Squared Error :  555.95547
 Root Mean Squared Error :  23.578707979870313
 R2 Score :  0.9721818902118338



1000_2000
 Mean Absolute Error :  18.078961038961033
 Mean Squared Error :  522.1622961038955
 Root Mean Squared Error :  22.850870795308776
 R2 Score :  0.9563906313127158





In [26]:
#This is the end-to-end pipelining and regression analysis of the Random Forest Classifier and Regressor.
#The code below runs the Random Forest Classifier and Regressor on the Flight Delay Prediction dataset.
#It first uses classification to predict whether a flight will be delayed or not, and then uses regression (only on those predicted as delayed) 
# to predict the delay time in minutes.

In [27]:
X_test_pipeline = X_test_classifier

X_train_pipeline = X_train_classifier

y_test_pipeline = y_test_classifier

y_train_classifier = y_train_classifier

In [28]:
clf.fit(X_test_pipeline,y_test_pipeline)

y_pred_pipeline = clf.predict(X_test_pipeline)

In [29]:
X_test_pipeline['Predicted'] = y_pred_pipeline

X_test_pipeline['ArrDelayMinutes'] = y_test_regressor

X_test_pipeline = X_test_pipeline[X_test_pipeline['Predicted'] == 1]

In [30]:
y_pipeline = X_test_pipeline['ArrDelayMinutes']

X_test_pipeline = X_test_pipeline.drop(['Predicted','ArrDelayMinutes'],axis = 1)

X_pipeline = X_test_pipeline

In [31]:
y_regressor_pipeline_pred = rfr.predict(X_pipeline)

In [32]:
print("\n\n Random Forest Regressor \n\n")

print(" Mean Absolute Error : ", metrics.mean_absolute_error(y_pipeline, y_regressor_pipeline_pred))  

print(" Mean Squared Error : ", metrics.mean_squared_error(y_pipeline, y_regressor_pipeline_pred))  

print(" Root Mean Squared Error : ", np.sqrt(metrics.mean_squared_error(y_pipeline, y_regressor_pipeline_pred)))

print(" R2 Score : ",metrics.r2_score(y_pipeline,y_regressor_pipeline_pred))



 Random Forest Regressor 


 Mean Absolute Error :  5.59800624015959
 Mean Squared Error :  94.55925648333621
 Root Mean Squared Error :  9.724158394603423
 R2 Score :  0.9800382634216509


In [33]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 1) Make sure your models are already trained on the TRAIN split:
#    clf.fit(X_train_classifier, y_train_classifier)
#    rfr.fit(X_train_regressor, y_train_regressor)

# 2) Define thresholds to evaluate
thresholds = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]

# 3) Prepare the clean test features (no extra columns)
# Only keep columns that were used during regressor training
X_feat = X_test_classifier[X_train_regressor.columns].copy()

# 4) Compute P(delay) for each test row
#    shape = (n_samples, 2), [:,1] is the probability of class '1'
proba = clf.predict_proba(X_feat)[:, 1]

# 5) Bring in the true continuous delays for ALL test rows, keyed by index
y_true_delays = y_test_regressor.copy()  # index matches X_test_classifier for rows where ArrDel15==1

# 6) Loop over thresholds
results = []
for i in thresholds:
    # 6a) select rows where P(delay) >= α
    invoke_mask = proba >= i
    invoke_idx  = X_feat.index[invoke_mask]
    invocation_rate = invoke_mask.mean()

    # 6b) Of those, *only* those that truly were delayed have a y_true_delays entry.
    #     So intersect with y_true_delays’s index:
    valid_idx = invoke_idx.intersection(y_true_delays.index)

    # 6c) Build Phase‑2 input/output
    # Ensure only the columns used during regressor training are selected
    X2 = X_feat.loc[valid_idx, X_train_regressor.columns]
    y2 = y_true_delays.loc[valid_idx, 'ArrDelayMinutes']

    # 6d) Predict delays only for those
    y2_pred = rfr.predict(X2)

    # 6e) Compute regression metrics
    mae  = mean_absolute_error(y2, y2_pred)
    rmse = np.sqrt(mean_squared_error(y2, y2_pred))
    r2   = r2_score(y2, y2_pred)

    results.append({
        'threshold': i,
        'invocation_rate': invocation_rate,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2
    })

# 7) Summarize results
df_results = pd.DataFrame(results)
print(df_results)


   threshold  invocation_rate       MAE       RMSE        R2
0        0.1         0.243244  5.597909   9.724053  0.980038
1        0.3         0.197463  5.597909   9.724053  0.980038
2        0.5         0.194849  5.597963   9.724118  0.980038
3        0.7         0.181174  5.881989  10.025225  0.979754
4        0.9         0.127830  7.241972  11.312469  0.979192
5        1.0         0.072750  7.519381  11.652355  0.977012
